<a href="https://colab.research.google.com/github/Mittapally-vani/Face-Emotion-Recognition/blob/main/face_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow opencv-python matplotlib


In [7]:
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Load the dataset
data = pd.read_csv('fer2013.csv')

# Prepare the data
def preprocess_data(data):
    X = []
    y = []
    for index, row in data.iterrows():
        # Convert pixel values from string to list of integers
        pixels = list(map(int, row['pixels'].split()))

        # Ensure the correct number of pixels (2304 for a 48x48 image)
        if len(pixels) == 48 * 48:
            image = np.array(pixels).reshape(48, 48)
            X.append(image)
            y.append(row['emotion'])
        else:
            print(f"Skipping row {index} due to incorrect number of pixels.")

    X = np.array(X)
    y = np.array(y)

    # Normalize pixel values
    X = X / 255.0

    # Convert labels to categorical format
    y = to_categorical(y, num_classes=7)  # FER2013 has 7 emotion classes

    # Reshape data to include the channel dimension
    X = X.reshape(X.shape[0], 48, 48, 1)

    return X, y

# Preprocess the data
X, y = preprocess_data(data)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


Skipping row 21998 due to incorrect number of pixels.


In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # 7 classes for 7 emotions

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Train the model
history = model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_test, y_test))


275/275 ━━━━━━━━━━━━━━━━━━━━ 81s 284ms/step - accuracy: 0.2394 - loss: 1.8372 - val_accuracy: 0.3291 - val_loss: 1.6861
Epoch 2/25
275/275 ━━━━━━━━━━━━━━━━━━━━ 71s 245ms/step - accuracy: 0.3323 - loss: 1.6795 - val_accuracy: 0.3939 - val_loss: 1.5380
Epoch 3/25
275/275 ━━━━━━━━━━━━━━━━━━━━ 84s 252ms/step - accuracy: 0.3896 - loss: 1.5586 - val_accuracy: 0.4348 - val_loss: 1.4519
Epoch 4/25
275/275 ━━━━━━━━━━━━━━━━━━━━ 79s 240ms/step - accuracy: 0.4308 - loss: 1.4704 - val_accuracy: 0.4707 - val_loss: 1.3858
Epoch 5/25
275/275 ━━━━━━━━━━━━━━━━━━━━ 82s 241ms/step - accuracy: 0.4635 - loss: 1.3880 - val_accuracy: 0.4825 - val_loss: 1.3457
Epoch 6/25
275/275 ━━━━━━━━━━━━━━━━━━━━ 82s 242ms/step - accuracy: 0.4829 - loss: 1.3600 - val_accuracy: 0.4914 - val_loss: 1.3192
Epoch 7/25
275/275 ━━━━━━━━━━━━━━━━━━━━ 82s 242ms/step - accuracy: 0.5058 - loss: 1.3067 - val_accuracy: 0.5080 - val_loss: 1.2803
Epoch 8/25
275/275 ━━━━━━━━━━━━━━━━━━━━ 85s 252ms/step - accuracy: 0.5235 - loss: 1.2607 - val

In [12]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc:.2f}')

# Save the model
# Save the model in the native Keras format
model.save('emotion_recognition_model.keras')



138/138 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - accuracy: 0.5239 - loss: 1.4670
Test accuracy: 0.53


In [16]:
# Load the saved model
model = tf.keras.models.load_model('emotion_recognition_model.keras')

# Function to predict emotion from an image
def predict_emotion(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (48, 48))
    image = image.reshape(1, 48, 48, 1) / 255.0
    prediction = model.predict(image)
    return np.argmax(prediction)

# Example usage
# emotion = predict_emotion(image)


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [19]:
# model has been created
# now to check the emotion of face, give an input image
# below are the steps to follow
import cv2
import numpy as np

# Function to preprocess the input image
def preprocess_image(angry):
    # Load the image
    image = cv2.imread(angry)

    # Convert to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Resize to 48x48 pixels
    resized_image = cv2.resize(gray_image, (48, 48))

    # Normalize pixel values to the range [0, 1]
    normalized_image = resized_image / 255.0

    # Reshape to match the input shape of the model: (1, 48, 48, 1)
    reshaped_image = normalized_image.reshape(1, 48, 48, 1)

    return reshaped_image


    import tensorflow as tf

# Load the saved model
model = tf.keras.models.load_model('emotion_recognition_model.keras')



/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 22 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [21]:
# Function to predict emotion from an image
def predict_emotion(image_path):
    # Preprocess the input image
    preprocessed_image = preprocess_image(image_path)

    # Predict the emotion
    prediction = model.predict(preprocessed_image)

    # Get the index of the emotion with the highest probability
    emotion_index = np.argmax(prediction)

    # Define the emotion labels (according to FER2013)
    emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

    # Return the predicted emotion
    return emotion_labels[emotion_index]

# Example usage
image_path = 'angry'
predicted_emotion = predict_emotion(image_path)
print(f'The predicted emotion is: {predicted_emotion}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
The predicted emotion is: Angry
